In [1]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from mne.preprocessing import ICA,read_ica

In [2]:
#read the ICA filtered data file from the folder
sample_data_raw_file=('C:/Users/user/Desktop/ICA/07062019 1_ica.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)
raw_temp=raw.copy().crop(689., 699.) #Copy the raw data and crop the 30 sec data basically skype call
raw_ch = raw_temp.pick_channels(['Fp1', 'F7', 'F3', 'FC1', 'FC2', 'F4', 'F8', 'Fp2']) # select only one channel

Opening raw data file C:/Users/user/Desktop/ICA/07062019 1_ica.fif...
    Range : 0 ... 242303 =      0.000 ...  1892.992 secs
Ready.
Reading 0 ... 242303  =      0.000 ...  1892.992 secs...


<ipython-input-2-30a2f0c2bbeb>:3: RuntimeWarning: This filename (C:/Users/user/Desktop/ICA/07062019 1_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)


In [3]:
raw_ch.info['ch_names']

['Fp1', 'F7', 'F3', 'FC1', 'FC2', 'F4', 'F8', 'Fp2']

In [4]:
Fp1_0 =raw_ch.copy().pick_channels(['Fp1']) 
F7_0 =raw_ch.copy().pick_channels(['F7'])
F3_0 =raw_ch.copy().pick_channels(['F3'])
FC1_0 =raw_ch.copy().pick_channels(['FC1'])
FC2_0 =raw_ch.copy().pick_channels(['FC2'])
F4_0 =raw_ch.copy().pick_channels(['F4'])
F8_0 =raw_ch.copy().pick_channels(['F8'])
Fp2_0 =raw_ch.copy().pick_channels(['Fp2'])

In [5]:
## Discret Wavelet Transform(time-frequency)
import pywt
def DWT( x ):
    resp = pywt.dwt(x, 'db4')
    return resp

def Energy(x):
    resp= abs(np.sum(x)**2)
    return resp
def BRMS(x):
    resp = np.sqrt(np.mean(x**2))
    return resp

def TKEO(x):
    tkeo = []

    for i in range(0, len(x)):
        if i == 0 or i == len(x) - 1:
            tkeo.append(x[i])
        else:
            tkeo.append((x[i]**2) - (x[i + 1] * x[i - 1]))
        tkeo =np.asarray(tkeo)
        resp = np.mean(tkeo,axis =1)
        #print('tkeo',tkeo)
        #print((tkeo.shape))
        return resp



In [6]:
# for channel Fp1
data_Fp1,times_w = Fp1_0[:]
A1_Fp1, D1_Fp1=DWT(data_Fp1)
#print(cA1)
A2_Fp1,D2_Fp1 = DWT(A1_Fp1)
A3_Fp1,D3_Fp1 = DWT(A2_Fp1)
#print(cA3) 
A4_Fp1,D4_Fp1 = DWT(A3_Fp1)
#print(cA4)
A5_Fp1,D5_Fp1 = DWT(A4_Fp1)
#print(cA5.shape)
#print(cA2)
#int(pywt.dwt_coeff_len(data_len=len(data_w), filter_len=w.dec_len, mode='symmetric'))

In [7]:
#Find energy of coefficient
theta_Fp1= Energy(D5_Fp1)
alpha_Fp1= Energy(D4_Fp1)
beta_Fp1 = Energy(D3_Fp1)
gamma_Fp1= Energy(D2_Fp1)

#total energy
total_Fp1 =np.sum([theta_Fp1,alpha_Fp1,beta_Fp1,gamma_Fp1])
#print('total',total)
#print('gamma',gamma)
# band energy ratio
th_rto_Fp1= theta_Fp1/total_Fp1
al_rto_Fp1= alpha_Fp1/total_Fp1
be_rto_Fp1= beta_Fp1/total_Fp1
ga_rto_Fp1= gamma_Fp1/total_Fp1
#########
#find rms of each band
rms_th_Fp1 =BRMS(theta_Fp1)
rms_al_Fp1 =BRMS(alpha_Fp1)
rms_be_Fp1 =BRMS(beta_Fp1)
rms_ga_Fp1 =BRMS(gamma_Fp1)
#finding tkeo mean for each band
th_tke_Fp1 = TKEO(D5_Fp1)
al_tke_Fp1 = TKEO(D4_Fp1)
be_tke_Fp1 = TKEO(D3_Fp1)
ga_tke_Fp1 = TKEO(D2_Fp1)

print((ga_tke_Fp1))

[-2.83439506e-08]


In [8]:
import pandas as pd 

data1 = [theta_Fp1]
data2 =[alpha_Fp1]
data3 = [beta_Fp1]
data4 = [gamma_Fp1]
data5 =[th_rto_Fp1]
data6 = [al_rto_Fp1]
data7 = [be_rto_Fp1]
data8 =[ga_rto_Fp1]
data9 = [rms_th_Fp1]
data10 =[rms_al_Fp1]
data11 = [rms_be_Fp1]
data12 =[rms_ga_Fp1]
data13 = [th_tke_Fp1]
data14 =[al_tke_Fp1]
data15 = [be_tke_Fp1]
data16 =[ga_tke_Fp1]
    
# List2  
#data_2 =  
    
# get the list of tuples from two lists.  
# and merge them by using zip().  
list_of_tuples = list(zip(data1, data2,data3, data4,data5, data6,data7, data8,data9, data10,
                         data11,data12,data13,data14,data15,data16,))  
    
# pandas Dataframe.  
df_Fp1 = pd.DataFrame(list_of_tuples, columns = ['E_theta', 'E_alpha','E_beta', 'E_gamma',
                                             'rto_theta', 'rto_alpha','rto_beta', 'rto_gamma',
                                             'rms_theta', 'rms_alpha','rms_beta', 'rms_gamma',
                                            'tke_theta', 'tke_alpha','tke_beta', 'tke_gamma',])  
     
# Print data.  
df_Fp1 

,E_theta,E_alpha,E_beta,E_gamma,rto_theta,rto_alpha,rto_beta,rto_gamma,rms_theta,rms_alpha,rms_beta,rms_gamma,tke_theta,tke_alpha,tke_beta,tke_gamma
0,1.024137e-08,4.401988e-09,2.142353e-10,8.485696e-11,0.685388,0.294596,0.014337,0.005679,1.024137e-08,4.401988e-09,2.142353e-10,8.485696e-11,[2.199992098308904e-06],[-7.714823295766025e-07],[8.81733675110627e-08],[-2.834395055962843e-08]


In [9]:
# for channel F7
data_F7,times_w = F7_0[:]
A1_F7, D1_F7=DWT(data_F7)
#print(cA1)
A2_F7,D2_F7 = DWT(A1_F7)
A3_F7,D3_F7 = DWT(A2_F7)
#print(cA3) 
A4_F7,D4_F7 = DWT(A3_F7)
#print(cA4)
A5_F7,D5_F7 = DWT(A4_F7)

In [10]:
#Find energy of coefficient
theta_F7= Energy(D5_F7)
alpha_F7= Energy(D4_F7)
beta_F7 = Energy(D3_F7)
gamma_F7= Energy(D2_F7)

#total energy
total_F7 =np.sum([theta_F7,alpha_F7,beta_F7,gamma_F7])
#print('total',total)
#print('gamma',gamma)
# band energy ratio
th_rto_F7= theta_F7/total_F7
al_rto_F7= alpha_F7/total_F7
be_rto_F7= beta_F7/total_F7
ga_rto_F7= gamma_F7/total_F7
#########
#find rms of each band
rms_th_F7 =BRMS(theta_F7)
rms_al_F7 =BRMS(alpha_F7)
rms_be_F7 =BRMS(beta_F7)
rms_ga_F7 =BRMS(gamma_F7)
#finding tkeo mean for each band
th_tke_F7 = TKEO(D5_F7)
al_tke_F7 = TKEO(D4_F7)
be_tke_F7 = TKEO(D3_F7)
ga_tke_F7 = TKEO(D2_F7)



In [11]:
import pandas as pd 

data1 = [theta_F7]
data2 =[alpha_F7]
data3 = [beta_F7]
data4 = [gamma_F7]
data5 =[th_rto_F7]
data6 = [al_rto_F7]
data7 = [be_rto_F7]
data8 =[ga_rto_F7]
data9 = [rms_th_F7]
data10 =[rms_al_F7]
data11 = [rms_be_F7]
data12 =[rms_ga_F7]
data13 = [th_tke_F7]
data14 =[al_tke_F7]
data15 = [be_tke_F7]
data16 =[ga_tke_F7]
    

# get the list of tuples from two lists.  
# and merge them by using zip().  
list_of_tuples = list(zip(data1, data2,data3, data4,data5, data6,data7, data8,data9, data10,
                         data11,data12,data13,data14,data15,data16,))  
    
# pandas Dataframe.  
df_F7 = pd.DataFrame(list_of_tuples, columns = ['E_theta', 'E_alpha','E_beta', 'E_gamma',
                                             'rto_theta', 'rto_alpha','rto_beta', 'rto_gamma',
                                             'rms_theta', 'rms_alpha','rms_beta', 'rms_gamma',
                                            'tke_theta', 'tke_alpha','tke_beta', 'tke_gamma',])  
     
# Print data.  
df_F7  

,E_theta,E_alpha,E_beta,E_gamma,rto_theta,rto_alpha,rto_beta,rto_gamma,rms_theta,rms_alpha,rms_beta,rms_gamma,tke_theta,tke_alpha,tke_beta,tke_gamma
0,1.159051e-07,1.155589e-07,6.420649e-10,8.147983e-09,0.482427,0.480986,0.002672,0.033914,1.159051e-07,1.155589e-07,6.420649e-10,8.147983e-09,[-7.401052052222751e-06],[3.9527858111995906e-06],[-1.5264457113038265e-07],[-2.777420877070924e-07]


In [12]:
# for channel F3
data_F3,times_w = F3_0[:]
A1_F3, D1_F3=DWT(data_F3)
#print(cA1)
A2_F3,D2_F3 = DWT(A1_F3)
A3_F3,D3_F3 = DWT(A2_F3)
#print(cA3) 
A4_F3,D4_F3 = DWT(A3_F3)
#print(cA4)
A5_F3,D5_F3 = DWT(A4_F3)

In [13]:
#Find energy of coefficient
theta_F3= Energy(D5_F3)
alpha_F3= Energy(D4_F3)
beta_F3 = Energy(D3_F3)
gamma_F3= Energy(D2_F3)

#total energy
total_F3 =np.sum([theta_F3,alpha_F3,beta_F3,gamma_F3])
#print('total',total)
#print('gamma',gamma)
# band energy ratio
th_rto_F3= theta_F3/total_F3
al_rto_F3= alpha_F3/total_F3
be_rto_F3= beta_F3/total_F3
ga_rto_F3= gamma_F3/total_F3
#########
#find rms of each band
rms_th_F3 =BRMS(theta_F3)
rms_al_F3 =BRMS(alpha_F3)
rms_be_F3 =BRMS(beta_F3)
rms_ga_F3 =BRMS(gamma_F3)
#finding tkeo mean for each band
th_tke_F3 = TKEO(D5_F3)
al_tke_F3 = TKEO(D4_F3)
be_tke_F3 = TKEO(D3_F3)
ga_tke_F3 = TKEO(D2_F3)


In [14]:
import pandas as pd 

data1 = [theta_F3]
data2 =[alpha_F3]
data3 = [beta_F3]
data4 = [gamma_F3]
data5 =[th_rto_F3]
data6 = [al_rto_F3]
data7 = [be_rto_F3]
data8 =[ga_rto_F3]
data9 = [rms_th_F3]
data10 =[rms_al_F3]
data11 = [rms_be_F3]
data12 =[rms_ga_F3]
data13 = [th_tke_F3]
data14 =[al_tke_F3]
data15 = [be_tke_F3]
data16 =[ga_tke_F3]
    

# get the list of tuples from two lists.  
# and merge them by using zip().  
list_of_tuples = list(zip(data1, data2,data3, data4,data5, data6,data7, data8,data9, data10,
                         data11,data12,data13,data14,data15,data16,))  
    
# pandas Dataframe.  
df_F3 = pd.DataFrame(list_of_tuples, columns = ['E_theta', 'E_alpha','E_beta', 'E_gamma',
                                             'rto_theta', 'rto_alpha','rto_beta', 'rto_gamma',
                                             'rms_theta', 'rms_alpha','rms_beta', 'rms_gamma',
                                            'tke_theta', 'tke_alpha','tke_beta', 'tke_gamma',])  
     
# Print data.  
df_F3  

,E_theta,E_alpha,E_beta,E_gamma,rto_theta,rto_alpha,rto_beta,rto_gamma,rms_theta,rms_alpha,rms_beta,rms_gamma,tke_theta,tke_alpha,tke_beta,tke_gamma
0,2.655842e-08,2.551393e-11,2.720480e-08,5.979213e-09,0.444359,0.000427,0.455174,0.10004,2.655842e-08,2.551393e-11,2.720480e-08,5.979213e-09,[-3.5427723411485595e-06],[5.8734092524227815e-08],[-9.936071682826198e-07],[-2.379242110556554e-07]


In [15]:
# for channel FC1
data_FC1,times_w = FC1_0[:]
A1_FC1, D1_FC1=DWT(data_FC1)
#print(cA1)
A2_FC1,D2_FC1 = DWT(A1_FC1)
A3_FC1,D3_FC1 = DWT(A2_FC1)
#print(cA3) 
A4_FC1,D4_FC1 = DWT(A3_FC1)
#print(cA4)
A5_FC1,D5_FC1 = DWT(A4_FC1)

In [16]:
#Find energy of coefficient
theta_FC1= Energy(D5_FC1)
alpha_FC1= Energy(D4_FC1)
beta_FC1 = Energy(D3_FC1)
gamma_FC1= Energy(D2_FC1)

#total energy
total_FC1 =np.sum([theta_FC1,alpha_FC1,beta_FC1,gamma_FC1])
#print('total',total)
#print('gamma',gamma)
# band energy ratio
th_rto_FC1= theta_FC1/total_FC1
al_rto_FC1= alpha_FC1/total_FC1
be_rto_FC1= beta_FC1/total_FC1
ga_rto_FC1= gamma_FC1/total_FC1
#########
#find rms of each band
rms_th_FC1 =BRMS(theta_FC1)
rms_al_FC1 =BRMS(alpha_FC1)
rms_be_FC1 =BRMS(beta_FC1)
rms_ga_FC1 =BRMS(gamma_FC1)
#finding tkeo mean for each band
th_tke_FC1 = TKEO(D5_FC1)
al_tke_FC1 = TKEO(D4_FC1)
be_tke_FC1 = TKEO(D3_FC1)
ga_tke_FC1 = TKEO(D2_FC1)


In [17]:
import pandas as pd 

data1 = [theta_FC1]
data2 =[alpha_FC1]
data3 = [beta_FC1]
data4 = [gamma_FC1]
data5 =[th_rto_FC1]
data6 = [al_rto_FC1]
data7 = [be_rto_FC1]
data8 =[ga_rto_FC1]
data9 = [rms_th_FC1]
data10 =[rms_al_FC1]
data11 = [rms_be_FC1]
data12 =[rms_ga_FC1]
data13 = [th_tke_FC1]
data14 =[al_tke_FC1]
data15 = [be_tke_FC1]
data16 =[ga_tke_FC1]
    

# get the list of tuples from two lists.  
# and merge them by using zip().  
list_of_tuples = list(zip(data1, data2,data3, data4,data5, data6,data7, data8,data9, data10,
                         data11,data12,data13,data14,data15,data16,))  
    
# pandas Dataframe.  
df_FC1 = pd.DataFrame(list_of_tuples, columns = ['E_theta', 'E_alpha','E_beta', 'E_gamma',
                                             'rto_theta', 'rto_alpha','rto_beta', 'rto_gamma',
                                             'rms_theta', 'rms_alpha','rms_beta', 'rms_gamma',
                                            'tke_theta', 'tke_alpha','tke_beta', 'tke_gamma',])  
     
# Print data.  
df_FC1  

,E_theta,E_alpha,E_beta,E_gamma,rto_theta,rto_alpha,rto_beta,rto_gamma,rms_theta,rms_alpha,rms_beta,rms_gamma,tke_theta,tke_alpha,tke_beta,tke_gamma
0,8.412036e-13,1.024174e-08,2.332902e-11,2.879340e-11,0.000082,0.994855,0.002266,0.002797,8.412036e-13,1.024174e-08,2.332902e-11,2.879340e-11,[1.993851117173387e-08],[1.1767612312881966e-06],[-2.9096458464876813e-08],[1.6510609065878605e-08]


In [18]:
# for channel FC1
data_FC2,times_w = FC2_0[:]
A1_FC2, D1_FC2=DWT(data_FC2)
#print(cA1)
A2_FC2,D2_FC2 = DWT(A1_FC2)
A3_FC2,D3_FC2 = DWT(A2_FC2)
#print(cA3) 
A4_FC2,D4_FC2 = DWT(A3_FC2)
#print(cA4)
A5_FC2,D5_FC2 = DWT(A4_FC2)

In [19]:
#Find energy of coefficient
theta_FC2= Energy(D5_FC2)
alpha_FC2= Energy(D4_FC2)
beta_FC2 = Energy(D3_FC2)
gamma_FC2= Energy(D2_FC2)

#total energy
total_FC2 =np.sum([theta_FC2,alpha_FC2,beta_FC2,gamma_FC2])
#print('total',total)
#print('gamma',gamma)
# band energy ratio
th_rto_FC2= theta_FC2/total_FC2
al_rto_FC2= alpha_FC2/total_FC2
be_rto_FC2= beta_FC2/total_FC2
ga_rto_FC2= gamma_FC2/total_FC2
#########
#find rms of each band
rms_th_FC2 =BRMS(theta_FC2)
rms_al_FC2 =BRMS(alpha_FC2)
rms_be_FC2 =BRMS(beta_FC2)
rms_ga_FC2 =BRMS(gamma_FC2)
#finding tkeo mean for each band
th_tke_FC2 = TKEO(D5_FC2)
al_tke_FC2 = TKEO(D4_FC2)
be_tke_FC2 = TKEO(D3_FC2)
ga_tke_FC2 = TKEO(D2_FC2)


In [20]:
import pandas as pd 

data1 = [theta_FC2]
data2 =[alpha_FC2]
data3 = [beta_FC2]
data4 = [gamma_FC2]
data5 =[th_rto_FC2]
data6 = [al_rto_FC2]
data7 = [be_rto_FC2]
data8 =[ga_rto_FC2]
data9 = [rms_th_FC2]
data10 =[rms_al_FC2]
data11 = [rms_be_FC2]
data12 =[rms_ga_FC2]
data13 = [th_tke_FC2]
data14 =[al_tke_FC2]
data15 = [be_tke_FC2]
data16 =[ga_tke_FC2]
    

# get the list of tuples from two lists.  
# and merge them by using zip().  
list_of_tuples = list(zip(data1, data2,data3, data4,data5, data6,data7, data8,data9, data10,
                         data11,data12,data13,data14,data15,data16,))  
    
# pandas Dataframe.  
df_FC2 = pd.DataFrame(list_of_tuples, columns = ['E_theta', 'E_alpha','E_beta', 'E_gamma',
                                             'rto_theta', 'rto_alpha','rto_beta', 'rto_gamma',
                                             'rms_theta', 'rms_alpha','rms_beta', 'rms_gamma',
                                            'tke_theta', 'tke_alpha','tke_beta', 'tke_gamma',])  
     
# Print data.  
df_FC2  

,E_theta,E_alpha,E_beta,E_gamma,rto_theta,rto_alpha,rto_beta,rto_gamma,rms_theta,rms_alpha,rms_beta,rms_gamma,tke_theta,tke_alpha,tke_beta,tke_gamma
0,3.446923e-09,4.039852e-09,2.905066e-11,2.857903e-09,0.332274,0.389431,0.0028,0.275494,3.446923e-09,4.039852e-09,2.905066e-11,2.857903e-09,[1.276315287725748e-06],[-7.390677609052225e-07],[-3.2469075148668435e-08],[1.644903245376423e-07]


In [21]:
# for channel F3
data_F4,times_w = F4_0[:]
A1_F4, D1_F4=DWT(data_F4)
#print(cA1)
A2_F4,D2_F4 = DWT(A1_F4)
A3_F4,D3_F4 = DWT(A2_F4)
#print(cA3) 
A4_F4,D4_F4 = DWT(A3_F4)
#print(cA4)
A5_F4,D5_F4 = DWT(A4_F4)

In [22]:
#Find energy of coefficient
theta_F4= Energy(D5_F4)
alpha_F4= Energy(D4_F4)
beta_F4 = Energy(D3_F4)
gamma_F4= Energy(D2_F4)

#total energy
total_F4 =np.sum([theta_F4,alpha_F4,beta_F4,gamma_F4])
#print('total',total)
#print('gamma',gamma)
# band energy ratio
th_rto_F4= theta_F4/total_F4
al_rto_F4= alpha_F4/total_F4
be_rto_F4= beta_F4/total_F4
ga_rto_F4= gamma_F4/total_F4
#########
#find rms of each band
rms_th_F4 =BRMS(theta_F4)
rms_al_F4 =BRMS(alpha_F4)
rms_be_F4 =BRMS(beta_F4)
rms_ga_F4 =BRMS(gamma_F4)
#finding tkeo mean for each band
th_tke_F4 = TKEO(D5_F4)
al_tke_F4 = TKEO(D4_F4)
be_tke_F4 = TKEO(D3_F4)
ga_tke_F4 = TKEO(D2_F4)


In [23]:
import pandas as pd 

data1 = [theta_F4]
data2 =[alpha_F4]
data3 = [beta_F4]
data4 = [gamma_F4]
data5 =[th_rto_F4]
data6 = [al_rto_F4]
data7 = [be_rto_F4]
data8 =[ga_rto_F4]
data9 = [rms_th_F4]
data10 =[rms_al_F4]
data11 = [rms_be_F4]
data12 =[rms_ga_F4]
data13 = [th_tke_F4]
data14 =[al_tke_F4]
data15 = [be_tke_F4]
data16 =[ga_tke_F4]
    

# get the list of tuples from two lists.  
# and merge them by using zip().  
list_of_tuples = list(zip(data1, data2,data3, data4,data5, data6,data7, data8,data9, data10,
                         data11,data12,data13,data14,data15,data16,))  
    
# pandas Dataframe.  
df_F4 = pd.DataFrame(list_of_tuples, columns = ['E_theta', 'E_alpha','E_beta', 'E_gamma',
                                             'rto_theta', 'rto_alpha','rto_beta', 'rto_gamma',
                                             'rms_theta', 'rms_alpha','rms_beta', 'rms_gamma',
                                            'tke_theta', 'tke_alpha','tke_beta', 'tke_gamma',])  
     
# Print data.  
df_F4  

,E_theta,E_alpha,E_beta,E_gamma,rto_theta,rto_alpha,rto_beta,rto_gamma,rms_theta,rms_alpha,rms_beta,rms_gamma,tke_theta,tke_alpha,tke_beta,tke_gamma
0,1.209342e-10,3.733960e-09,6.299086e-09,1.280222e-10,0.011762,0.363155,0.612632,0.012451,1.209342e-10,3.733960e-09,6.299086e-09,1.280222e-10,[2.39065434057977e-07],[-7.10536488922527e-07],[4.781131530459905e-07],[3.481443097879017e-08]


In [24]:
# for channel F3
data_F8,times_w = F8_0[:]
A1_F8, D1_F8=DWT(data_F8)
#print(cA1)
A2_F8,D2_F8 = DWT(A1_F8)
A3_F8,D3_F8 = DWT(A2_F8)
#print(cA3) 
A4_F8,D4_F8 = DWT(A3_F8)
#print(cA4)
A5_F8,D5_F8 = DWT(A4_F8)

In [25]:
#Find energy of coefficient
theta_F8= Energy(D5_F8)
alpha_F8= Energy(D4_F8)
beta_F8 = Energy(D3_F8)
gamma_F8= Energy(D2_F8)

#total energy
total_F8 =np.sum([theta_F8,alpha_F8,beta_F8,gamma_F8])
#print('total',total)
#print('gamma',gamma)
# band energy ratio
th_rto_F8= theta_F8/total_F8
al_rto_F8= alpha_F8/total_F8
be_rto_F8= beta_F8/total_F8
ga_rto_F8= gamma_F8/total_F8
#########
#find rms of each band
rms_th_F8 =BRMS(theta_F8)
rms_al_F8 =BRMS(alpha_F8)
rms_be_F8 =BRMS(beta_F8)
rms_ga_F8 =BRMS(gamma_F8)
#finding tkeo mean for each band
th_tke_F8 = TKEO(D5_F8)
al_tke_F8 = TKEO(D4_F8)
be_tke_F8 = TKEO(D3_F8)
ga_tke_F8 = TKEO(D2_F8)


In [26]:
import pandas as pd 

data1 = [theta_F8]
data2 =[alpha_F8]
data3 = [beta_F8]
data4 = [gamma_F8]
data5 =[th_rto_F8]
data6 = [al_rto_F8]
data7 = [be_rto_F8]
data8 =[ga_rto_F8]
data9 = [rms_th_F8]
data10 =[rms_al_F8]
data11 = [rms_be_F8]
data12 =[rms_ga_F8]
data13 = [th_tke_F8]
data14 =[al_tke_F8]
data15 = [be_tke_F8]
data16 =[ga_tke_F8]
    

# get the list of tuples from two lists.  
# and merge them by using zip().  
list_of_tuples = list(zip(data1, data2,data3, data4,data5, data6,data7, data8,data9, data10,
                         data11,data12,data13,data14,data15,data16,))  
    
# pandas Dataframe.  
df_F8 = pd.DataFrame(list_of_tuples, columns = ['E_theta', 'E_alpha','E_beta', 'E_gamma',
                                             'rto_theta', 'rto_alpha','rto_beta', 'rto_gamma',
                                             'rms_theta', 'rms_alpha','rms_beta', 'rms_gamma',
                                            'tke_theta', 'tke_alpha','tke_beta', 'tke_gamma',])  
     
# Print data.  
df_F8  

,E_theta,E_alpha,E_beta,E_gamma,rto_theta,rto_alpha,rto_beta,rto_gamma,rms_theta,rms_alpha,rms_beta,rms_gamma,tke_theta,tke_alpha,tke_beta,tke_gamma
0,1.461837e-08,3.633758e-08,4.044034e-09,1.901580e-08,0.197503,0.490944,0.054637,0.256915,1.461837e-08,3.633758e-08,4.044034e-09,1.901580e-08,[2.6284009316809647e-06],[-2.216560164723513e-06],[-3.830886670922313e-07],[-4.243008327597735e-07]


In [27]:
# for channel Fp1
data_Fp2,times_w = Fp2_0[:]
A1_Fp2, D1_Fp2=DWT(data_Fp2)
#print(cA1)
A2_Fp2,D2_Fp2 = DWT(A1_Fp2)
A3_Fp2,D3_Fp2 = DWT(A2_Fp2)
#print(cA3) 
A4_Fp2,D4_Fp2 = DWT(A3_Fp2)
#print(cA4)
A5_Fp2,D5_Fp2 = DWT(A4_Fp2)
#print(cA5.shape)

In [28]:
#Find energy of coefficient
theta_Fp2= Energy(D5_Fp2)
alpha_Fp2= Energy(D4_Fp2)
beta_Fp2 = Energy(D3_Fp2)
gamma_Fp2= Energy(D2_Fp2)

#total energy
total_Fp2 =np.sum([theta_Fp2,alpha_Fp2,beta_Fp2,gamma_Fp2])
#print('total',total)
#print('gamma',gamma)
# band energy ratio
th_rto_Fp2= theta_Fp2/total_Fp2
al_rto_Fp2= alpha_Fp2/total_Fp2
be_rto_Fp2= beta_Fp2/total_Fp2
ga_rto_Fp2= gamma_Fp2/total_Fp2
#########
#find rms of each band
rms_th_Fp2 =BRMS(theta_Fp2)
rms_al_Fp2 =BRMS(alpha_Fp2)
rms_be_Fp2 =BRMS(beta_Fp2)
rms_ga_Fp2 =BRMS(gamma_Fp2)
#finding tkeo mean for each band
th_tke_Fp2 = TKEO(D5_Fp2)
al_tke_Fp2 = TKEO(D4_Fp2)
be_tke_Fp2 = TKEO(D3_Fp2)
ga_tke_Fp2 = TKEO(D2_Fp2)


In [29]:
import pandas as pd 

data1 = [theta_Fp2]
data2 =[alpha_Fp2]
data3 = [beta_Fp2]
data4 = [gamma_Fp2]
data5 =[th_rto_Fp2]
data6 = [al_rto_Fp2]
data7 = [be_rto_Fp2]
data8 =[ga_rto_Fp2]
data9 = [rms_th_Fp2]
data10 =[rms_al_Fp2]
data11 = [rms_be_Fp2]
data12 =[rms_ga_Fp2]
data13 = [th_tke_Fp2]
data14 =[al_tke_Fp2]
data15 = [be_tke_Fp2]
data16 =[ga_tke_Fp2]
    
# List2  
#data_2 =  
    
# get the list of tuples from two lists.  
# and merge them by using zip().  
list_of_tuples = list(zip(data1, data2,data3, data4,data5, data6,data7, data8,data9, data10,
                         data11,data12,data13,data14,data15,data16,))  
    
# pandas Dataframe.  
df_Fp2 = pd.DataFrame(list_of_tuples, columns = ['E_theta', 'E_alpha','E_beta', 'E_gamma',
                                             'rto_theta', 'rto_alpha','rto_beta', 'rto_gamma',
                                             'rms_theta', 'rms_alpha','rms_beta', 'rms_gamma',
                                            'tke_theta', 'tke_alpha','tke_beta', 'tke_gamma',])  
     
# Print data.  
df_Fp2 

,E_theta,E_alpha,E_beta,E_gamma,rto_theta,rto_alpha,rto_beta,rto_gamma,rms_theta,rms_alpha,rms_beta,rms_gamma,tke_theta,tke_alpha,tke_beta,tke_gamma
0,1.076167e-08,2.079990e-09,2.467361e-11,3.421891e-09,0.660702,0.127699,0.001515,0.210084,1.076167e-08,2.079990e-09,2.467361e-11,3.421891e-09,[2.255183878211064e-06],[5.303128283756605e-07],[2.992321478308277e-08],[1.7999055460445964e-07]


In [30]:
ch_name =['Fp1', 'F7', 'F3', 'FC1', 'FC2', 'F4', 'F8', 'Fp2']
result = pd.concat([df_Fp1 ,df_F7,df_F3,df_FC1,df_FC2,df_F4, df_F8,df_Fp2 ],
                       sort=False)
result = result.assign(**{'ch_name':ch_name,'phase': 1})
result.set_index("phase", inplace = True)               
result

,E_theta,E_alpha,E_beta,E_gamma,rto_theta,rto_alpha,rto_beta,rto_gamma,rms_theta,rms_alpha,rms_beta,rms_gamma,tke_theta,tke_alpha,tke_beta,tke_gamma,ch_name
phase,,,,,,,,,,,,,,,,,
1,1.024137e-08,4.401988e-09,2.142353e-10,8.485696e-11,0.685388,0.294596,0.014337,0.005679,1.024137e-08,4.401988e-09,2.142353e-10,8.485696e-11,[2.199992098308904e-06],[-7.714823295766025e-07],[8.81733675110627e-08],[-2.834395055962843e-08],Fp1
1,1.159051e-07,1.155589e-07,6.420649e-10,8.147983e-09,0.482427,0.480986,0.002672,0.033914,1.159051e-07,1.155589e-07,6.420649e-10,8.147983e-09,[-7.401052052222751e-06],[3.9527858111995906e-06],[-1.5264457113038265e-07],[-2.777420877070924e-07],F7
1,2.655842e-08,2.551393e-11,2.720480e-08,5.979213e-09,0.444359,0.000427,0.455174,0.100040,2.655842e-08,2.551393e-11,2.720480e-08,5.979213e-09,[-3.5427723411485595e-06],[5.8734092524227815e-08],[-9.936071682826198e-07],[-2.379242110556554e-07],F3
1,8.412036e-13,1.024174e-08,2.332902e-11,2.879340e-11,0.000082,0.994855,0.002266,0.002797,8.412036e-13,1.024174e-08,2.332902e-11,2.879340e-11,[1.993851117173387e-08],[1.1767612312881966e-06],[-2.9096458464876813e-08],[1.6510609065878605e-08],FC1
1,3.446923e-09,4.039852e-09,2.905066e-11,2.857903e-09,0.332274,0.389431,0.002800,0.275494,3.446923e-09,4.039852e-09,2.905066e-11,2.857903e-09,[1.276315287725748e-06],[-7.390677609052225e-07],[-3.2469075148668435e-08],[1.644903245376423e-07],FC2
1,1.209342e-10,3.733960e-09,6.299086e-09,1.280222e-10,0.011762,0.363155,0.612632,0.012451,1.209342e-10,3.733960e-09,6.299086e-09,1.280222e-10,[2.39065434057977e-07],[-7.10536488922527e-07],[4.781131530459905e-07],[3.481443097879017e-08],F4
1,1.461837e-08,3.633758e-08,4.044034e-09,1.901580e-08,0.197503,0.490944,0.054637,0.256915,1.461837e-08,3.633758e-08,4.044034e-09,1.901580e-08,[2.6284009316809647e-06],[-2.216560164723513e-06],[-3.830886670922313e-07],[-4.243008327597735e-07],F8
1,1.076167e-08,2.079990e-09,2.467361e-11,3.421891e-09,0.660702,0.127699,0.001515,0.210084,1.076167e-08,2.079990e-09,2.467361e-11,3.421891e-09,[2.255183878211064e-06],[5.303128283756605e-07],[2.992321478308277e-08],[1.7999055460445964e-07],Fp2


In [31]:
result.to_csv('C:/Users/user/Desktop/Instred_feature/time-frequency/07062019/10 second/phase 1.csv')